# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
print(full_data_rows_list[:3])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91'], ['Manowar', 'Logged In', 'Jacob', 'M', '0', 'Klein', '247.562', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Shell Shock', '200', '1.54354E+12', '73'], ['Morcheeba', 'Logged In', 'Jacob', 'M', '1', 'Klein', '257.41016', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '1049', 'Women Lose Weight (Feat: Slick Rick)', '200', '1.54354E+12', '73']]


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra data modelling

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Creating tables to get the following data. 

- 1. Artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
-  2. Name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- 3. User name (first and last) in the music app history who listened to the song 'All Hands Against His Own'




<img src="images/image_event_datafile_new.jpg">

In [8]:
df_event_datafile_new = pd.read_csv('event_datafile_new.csv')
df_event_datafile_new.head(2)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91
1,Manowar,Jacob,M,0,Klein,247.56200,paid,"Tampa-St. Petersburg-Clearwater, FL",1049,Shell Shock,73


##### Query 1

In [ ]:
_ = session.execute("DROP TABLE IF EXISTS artist_song__session_id_item")

In [102]:
query = "CREATE TABLE IF NOT EXISTS artist_song__session_id_item"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, \
                PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [51]:
for i, row in df_event_datafile_new.iterrows():
        query = "INSERT INTO artist_song__session_id_item (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (row['artist'], row['song'], row['length'], row['sessionId'], row['itemInSession']))


##### SELECT to verify that the data have been inserted into each table

In [52]:
query = "select artist, song, length from artist_song__session_id_item WHERE sessionId=338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
list(rows)

[Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)]

##### Query 2

In [58]:
df_event_datafile_new.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91


In [68]:
_ = session.execute("DROP TABLE IF EXISTS artist_song__user_session_id")

In [69]:
query = "CREATE TABLE IF NOT EXISTS artist_song__user_session_id"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, \
                lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)        

In [70]:
for i, row in df_event_datafile_new.iterrows():
        query = "INSERT INTO artist_song__user_session_id (artist, song, firstName, lastName, userId, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (row['artist'], row['song'], row['firstName'], 
                                row['lastName'], row['userId'], row['sessionId'], row['itemInSession']))


In [72]:
query = "select artist, song, firstName, lastName from artist_song__user_session_id \
        WHERE userId=10 and sessionId=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
list(rows)

[Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz', iteminsession=0),
 Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz', iteminsession=1),
 Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz', iteminsession=2),
 Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', iteminsession=3)]

##### Query 3

In [93]:
_ = session.execute("DROP TABLE IF EXISTS user_name__song_session_id")

In [94]:
query = "CREATE TABLE IF NOT EXISTS user_name__song_session_id"
query = query + "(song text, userId int, sessionId int, firstName text, lastName text, \
                PRIMARY KEY ((song), userId, sessionId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [95]:
for i, row in df_event_datafile_new.iterrows():
    query = "INSERT INTO user_name__song_session_id (song, userId, sessionId, firstName, lastName)"
    query = query + " VALUES (%s, %s, %s, %s, %s)"
    session.execute(query, (row['song'],row['userId'], row['sessionId'], row['firstName'], row['lastName']))

In [97]:
query = "select firstName, lastName from user_name__song_session_id \
        WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
list(rows)

[Row(firstname='Jacqueline', lastname='Lynch'),
 Row(firstname='Tegan', lastname='Levine'),
 Row(firstname='Sara', lastname='Johnson')]

### Drop the tables before closing out the sessions

In [99]:
query = "drop table artist_song__session_id_item"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [100]:
query = "drop table artist_song__user_session_id"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [101]:
query = "drop table user_name__song_session_id"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [22]:
session.shutdown()
cluster.shutdown()